In [1]:
import os
import pandas as pd
from Bio import SeqIO, Entrez
import re
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import scipy.spatial
import random
import pickle
os.chdir('/Users/callamartyn/chou_lab/alongshan/')

In [16]:
taxdict = {}
org_tax={}
for org in queries:
    id_handle = Entrez.esearch(db='taxonomy', term=org, retmode="xml")
    o=Entrez.read(id_handle)
    if o['Count']=='0':
            continue
    else:
        taxid = o['IdList'][0]
    org_tax[org]=taxid
    handle = Entrez.efetch(db='taxonomy', id=str(taxid), retmode="xml")
    record = Entrez.read(handle)
    id_dict = {}
    for x in record[0]['LineageEx']:
        id_dict[x['Rank']] = x['ScientificName']
    taxdict[taxid] = id_dict

In [2]:

def get_taxinfo(fasta):
    input_seq_iterator=SeqIO.parse(fasta, 'fasta')
    names=[record.description for record in input_seq_iterator]
    queries=[s[s.find("[")+1:s.find("]")] for s in names]
    for org in queries:
        if org in org_tax:
            continue
        id_handle = Entrez.esearch(db='taxonomy', term=org, retmode="xml")
        o=Entrez.read(id_handle)
        if o['Count']=='0':
                continue
        else:
            taxid = o['IdList'][0]
        org_tax[org]=taxid
        handle = Entrez.efetch(db='taxonomy', id=str(taxid), retmode="xml")
        record = Entrez.read(handle)
        id_dict = {}
        for x in record[0]['LineageEx']:
            id_dict[x['Rank']] = x['ScientificName']
        taxdict[taxid] = id_dict

In [3]:
org_tax={}
taxdict={}
# get_taxinfo('./out/trees/notleys_landing_ref_dedup.fasta')
# get_taxinfo('./out/trees/bunyavirales_rdrp_80_dedup.fasta')
# get_taxinfo('./out/trees/Cabrillo_virus_95_dedup.fasta')
# get_taxinfo('./out/trees/Castle_Rock_Virus_95_dedup.fasta')
# get_taxinfo('./out/trees/bunyavirales_combined.fasta')
# get_taxinfo('./out/trees/Garrapata_Virus.fasta')
# get_taxinfo('./out/trees/monjiviricetes.fasta')
# get_taxinfo('./out/trees/Notleys_Landing_Virus_str1_95_dedup.fasta')
# get_taxinfo('./out/trees/narnaviridae_95_dedup.fasta')
get_taxinfo('./out/trees/rhabdoviridae_ref.fasta')

/Users/callamartyn/opt/miniconda3/lib/python3.8/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


In [ ]:
import pickle
keys_values = org_tax.items()
newd={str(key): str(value) for key, value in keys_values}

pickle.dump(newd, open('taxids.pickle', 'wb'))



In [14]:
id_dict = pickle.load(open('taxids.pickle', 'rb'))
tax_dict = pickle.load(open('taxonomy.pickle', 'rb'))
keys_values = tax_dict.items()
tax_dict={key: eval(value) for key, value in keys_values}

In [4]:
id_dict=org_tax
tax_dict=taxdict

In [5]:
def get_clade(fasta):
    input_seq_iterator=SeqIO.parse(fasta, 'fasta')
    names=[record.description for record in input_seq_iterator]
    queries=[s[s.find("[")+1:s.find("]")] for s in names]
   
    family=[]
    genus=[]
    no_rank=[]
    subfamily=[]
    for n in queries:
        t=id_dict.get(n)
        if t == None:
            family.append(None)
            no_rank.append(None)
            genus.append(None)
            subfamily.append(None)
        else:
            family.append(tax_dict[t].get('family'))
            no_rank.append(tax_dict[t].get('no rank'))
            genus.append(tax_dict[t].get('genus'))
            subfamily.append(tax_dict[t].get('subfamily'))
    df=pd.DataFrame({'virus_name':queries, 'family':family, 'genus':genus, 'no_rank':no_rank, 'subfamily':subfamily})
    return(df)

In [11]:
pal=[(0.8280838378584804, 0.2317149736053096, 0.19371846323785552),
     (0.8280838378584804, 0.2317149736053096, 0.49371846323785552),
     (0.9677975592919913, 0.24127456009157356, 0.2358103155058701),
      (0.9677975592919913, 0.44127456009157356, 0.5358103155058701), 
      (0.9218528551477547, 0.500345433264, 0.19606247053399534),
     (0.9218528551477547, 0.6500345433264, 0.19606247053399534), 
     (0.856593795426372, 0.78660455674609, 0.19460548454202936), 
     (0.9280838378584804, 0.8317149736053096, 0.01371846323785552),
      (0.6280838378584804, 0.8317149736053096, 0.19371846323785552),
      (0.14127702936426016, 0.4747986506725627, 0.09278677787286805), 
      (0.19925855442314272, 0.6935861010390585, 0.4349484671340832), 
      (0.20730980534421067, 0.8820252887361699, 0.5931178995944008), 
      (0.213603436221035, 0.5124447149771248, 0.9800417229996091), 
      (0.2218708221705632, 0.6590814541823624, 0.7990680761766646), 
      (0.38670894369284947, 0.6148978276981065, 0.9585912355668605), 
      (0.7049741506310939, 0.5248938690186019, 0.9581527270311664),
     (0.5280838378584804, 0.2317149736053096, 0.39371846323785552),
      (0.7049741506310939, 0.408938690186019, 0.7381527270311664), 
      (0.9538763597142671, 0.7594595647605631, 0.95756585161213), 
      (0.9637107389395935, 0.7595260281088575, 0.7436447221919831)]
sns.color_palette(palette=pal)
random.seed(18)
colors=sns.color_palette(palette=random.sample(pal,len(pal))).as_hex()
# colors=colors.as_hex()

In [12]:
pd.set_option('display.max_rows', 500)
df=get_clade('./out/trees/rhabdoviridae_ref.fasta')
df['no_rank']=df['no_rank'].fillna(df.genus)
df['no_rank']=df.no_rank.str.replace('unclassified ', '')
len(df.no_rank.unique())
df.insert(1, 'range','range')
color_dict=dict(zip(list(df.no_rank.unique()), colors[0:len(df.no_rank.unique())]))
df.insert(2, 'color', df.no_rank.map(color_dict))
df[['virus_name', 'range', 'color', 'no_rank']].to_csv(
    './out/trees/rhabdoviridae_annotation.txt', index=False, sep= '\t', header=False)


In [13]:
df

,virus_name,range,color,family,genus,no_rank,subfamily
0,Doud Peak Virus,range,#eba632,None,None,None,None
1,Norway mononegavirus 1,range,#f77189,None,None,Mononegavirales,None
2,Norway mononegavirus 1,range,#f77189,None,None,Mononegavirales,None
3,Mononegavirales sp.,range,#f77189,None,None,Mononegavirales,None
4,Manly virus,range,#639df4,None,None,Riboviria,None
5,Huangpi Tick Virus 3,range,#33b16f,Rhabdoviridae,None,Rhabdoviridae,None
6,Tacheng Tick Virus 3,range,#33b16f,Rhabdoviridae,None,Rhabdoviridae,None
7,Bole Tick Virus 2,range,#edd403,Rhabdoviridae,Alpharicinrhavirus,Alpharicinrhavirus,Alpharhabdovirinae
8,Bole Tick Virus 2,range,#edd403,Rhabdoviridae,Alpharicinrhavirus,Alpharicinrhavirus,Alpharhabdovirinae
9,Bole Tick Virus 2,range,#edd403,Rhabdoviridae,Alpharicinrhavirus,Alpharicinrhavirus,Alpharhabdovirinae


In [30]:
df=get_clade('./out/trees/notleys_landing_ref_dedup.fasta')
df.insert(1, 'range','range')
color_dict=dict(zip(list(df.no_rank.unique()), colors[0:len(df.no_rank.unique())]))
df.insert(2, 'color', df.no_rank.map(color_dict))
df[['virus_name', 'range', 'color', 'no_rank']].to_csv(
    './out/trees/notleys_landing_virus_annotation.txt', index=False, sep= '\t', header=False)


In [102]:
df=get_clade('./out/trees/bunyavirales_rdrp_80_dedup.fasta')
df.insert(1, 'range','range')
df['no_rank']=df['no_rank'].fillna(df.genus)
df['no_rank']=df.no_rank.str.replace('unclassified ', '')
color_dict=dict(zip(list(df.no_rank.unique()), p[0:len(df.no_rank.unique())]))
df.insert(2, 'color', df.no_rank.map(color_dict))
df[['virus_name', 'range', 'color', 'no_rank']].to_csv(
    './out/trees/bunyavirales_annotation.txt', index=False, sep= '\t', header=False)


In [95]:
pal1=sns.hls_palette(12, l=.3).as_hex()
pal2=sns.hls_palette(12, l=.5).as_hex()
pal3=sns.hls_palette(12, l=.8).as_hex()
p=pal1+pal2+pal3


In [103]:
df.no_rank

0                  None
1                  None
2                  None
3        Mammarenavirus
4       Orthobunyavirus
5       Orthobunyavirus
6             Coguvirus
7           Phlebovirus
8        Mammarenavirus
9        Mammarenavirus
10       Mammarenavirus
11          Phlebovirus
12     Orthophasmavirus
13            Uukuvirus
14          Phlebovirus
15          Phlebovirus
16          Phlebovirus
17          Phlebovirus
18          Phlebovirus
19            Uukuvirus
20          Phlebovirus
21          Phlebovirus
22          Phlebovirus
23         Agnathovirus
24          Actinovirus
25          Actinovirus
26        Reptillovirus
27      Orthotospovirus
28      Orthohantavirus
29          Phlebovirus
30         Bunyavirales
31       Mammarenavirus
32      Orthonairovirus
33         Tospoviridae
34            Feravirus
35         Bunyavirales
36           Phasivirus
37        Phasmaviridae
38        Phenuiviridae
39          Phlebovirus
40      Orthonairovirus
41      Orthonai

In [84]:
df=get_clade('./out/trees/Cabrillo_virus_95_dedup.fasta')
df.insert(1, 'range','range')
color_dict=dict(zip(list(df.genus.unique()), colors[0:len(df.genus.unique())]))
df.insert(2, 'color', df.genus.map(color_dict))
df[['virus_name', 'range', 'color', 'genus']].to_csv(
    './out/trees/Cabrillo_Virus_annotation.txt', index=False, sep= '\t', header=False)


In [91]:
df=get_clade('./out/trees/Castle_Rock_Virus_95_dedup.fasta')
df.insert(1, 'range','range')
color_dict=dict(zip(list(df.genus.unique()), colors[0:len(df.genus.unique())]))
df.insert(2, 'color', df.genus.map(color_dict))
df[['virus_name', 'range', 'color', 'genus']].to_csv(
    './out/trees/Castle_Rock_Virus_annotation.txt', index=False, sep= '\t', header=False)

In [176]:
df=get_clade('./out/trees/Portuguese_Ridge_virus_95_dedup.fasta')
df['clade']=np.where(df['genus'].isnull(),df['no_rank'],df['genus'])
df.insert(1, 'range','range')
color_dict=dict(zip(list(df.clade.unique()), colors[0:len(df.clade.unique())]))
df.insert(2, 'color', df.clade.map(color_dict))
df[['virus_name', 'range', 'color', 'clade']].to_csv(
    './out/trees/Portuguese_Ridge_Virus_annotation.txt', index=False, sep= '\t', header=False)

In [8]:
def write_annotation(fasta, outname):
    df=get_clade(fasta)
    df['clade']=np.where(df['genus'].isnull(),df['no_rank'],df['genus'])
    df.insert(1, 'range','range')
    color_dict=dict(zip(list(df.clade.unique()), colors[0:len(df.clade.unique())]))
    df.insert(2, 'color', df.clade.map(color_dict))
    df[['virus_name', 'range', 'color', 'clade']].to_csv(
    './out/trees/' + outname+'.txt', index=False, sep= '\t', header=False)
    return df

In [182]:
write_annotation('./out/trees/Castle_Rock_Virus_95_dedup.fasta', 'Castle_Rock_Virus')

,virus_name,range,color,family,genus,no_rank,clade
0,Caste Rock Virus,range,#fcfffd,None,None,None,None
1,Norway partiti-like virus 1,range,#a1c9f4,Partitiviridae,None,unclassified Partitiviridae,unclassified Partitiviridae
2,Jilin partiti-like virus 1,range,#a1c9f4,Partitiviridae,None,unclassified Partitiviridae,unclassified Partitiviridae
3,North shore virus,range,#ffb482,None,None,unclassified Riboviria,unclassified Riboviria
4,Hubei partiti-like virus 57,range,#8de5a1,None,None,unclassified RNA viruses ShiM-2016,unclassified RNA viruses ShiM-2016
5,Culex pseudovishnui partitivirus,range,#a1c9f4,Partitiviridae,None,unclassified Partitiviridae,unclassified Partitiviridae
6,Hubei partiti-like virus 56,range,#8de5a1,None,None,unclassified RNA viruses ShiM-2016,unclassified RNA viruses ShiM-2016
7,Changjiang partiti-like virus 1,range,#8de5a1,None,None,unclassified RNA viruses ShiM-2016,unclassified RNA viruses ShiM-2016
8,Beihai partiti-like virus 11,range,#8de5a1,None,None,unclassified RNA viruses ShiM-2016,unclassified RNA viruses ShiM-2016
9,Aphis gossypii,range,#ff9f9b,Aphididae,Aphis,cellular organisms,Aphis


In [183]:
write_annotation('./out/trees/Notleys_Landing_Virus_str1_95_dedup.fasta', 'Notleys_Landing_Virus')

,virus_name,range,color,family,genus,no_rank,clade
0,Notely's Landing Virus str 1,range,#fcfffd,None,None,None,None
1,Notley's Landing Virus str 2,range,#fcfffd,None,None,None,None
2,Ixodes scapularis associated virus 1,range,#a1c9f4,None,None,unclassified viruses,unclassified viruses
3,Norway luteo-like virus 3,range,#ffb482,Luteoviridae,None,unclassified Luteoviridae,unclassified Luteoviridae
4,Xinjiang tick associated virus 1,range,#a1c9f4,None,None,unclassified viruses,unclassified viruses
5,American dog tick associated virus-1,range,#a1c9f4,None,None,unclassified viruses,unclassified viruses
6,Store beach virus,range,#8de5a1,None,None,unclassified Riboviria,unclassified Riboviria
7,Ixodes scapularis associated virus 2,range,#a1c9f4,None,None,unclassified viruses,unclassified viruses
8,Norway luteo-like virus 2,range,#ffb482,Luteoviridae,None,unclassified Luteoviridae,unclassified Luteoviridae
9,Jilin luteo-like virus 2,range,#ffb482,Luteoviridae,None,unclassified Luteoviridae,unclassified Luteoviridae


In [187]:
write_annotation('./out/trees/Garrapata_Virus_95_dedup.fasta', 'Garrapata_Virus')

,virus_name,range,color,family,genus,no_rank,clade
0,Garrapata Virus,range,#fcfffd,None,None,None,None
1,Alongshan virus,range,#a1c9f4,Flaviviridae,None,unclassified Flaviviridae,unclassified Flaviviridae
2,Jingmen tick virus,range,#a1c9f4,Flaviviridae,None,unclassified Flaviviridae,unclassified Flaviviridae
3,Xinjiang tick virus 1,range,#ffb482,None,None,unclassified viruses,unclassified viruses
4,Yanggou tick virus,range,#ffb482,None,None,unclassified viruses,unclassified viruses
5,Flavi-like segmented virus,range,#a1c9f4,Flaviviridae,None,unclassified Flaviviridae,unclassified Flaviviridae
6,Kindia tick virus,range,#8de5a1,None,None,unclassified ssRNA viruses,unclassified ssRNA viruses
7,Plasmopara viticola lesion associated Jingman-...,range,#a1c9f4,Flaviviridae,None,unclassified Flaviviridae,unclassified Flaviviridae
8,Soybean thrips virus 3,range,#ff9f9b,None,None,unclassified Riboviria,unclassified Riboviria
9,Wuhan aphid virus 1,range,#8de5a1,None,None,unclassified ssRNA viruses,unclassified ssRNA viruses


In [185]:
write_annotation('./out/trees/bunyavirales_combined.fasta', 'bunyavirales')

,virus_name,range,color,family,genus,no_rank,clade
0,Painters Point Virus,range,#fcfffd,None,None,None,None
1,Bronnoya virus,range,#a1c9f4,None,None,unclassified Bunyavirales,unclassified Bunyavirales
2,Ubmeje virus,range,#a1c9f4,None,None,unclassified Bunyavirales,unclassified Bunyavirales
3,Volzhskoe tick virus,range,#a1c9f4,None,None,unclassified Bunyavirales,unclassified Bunyavirales
4,Plecopteran hanta-related virus OKIAV215,range,#ffb482,Hantaviridae,None,unclassified Hantaviridae,unclassified Hantaviridae
5,Frankliniella occidentalis associated peribuny...,range,#8de5a1,Peribunyaviridae,None,unclassified Peribunyaviridae,unclassified Peribunyaviridae
6,Ness Ziona virus,range,#ff9f9b,Peribunyaviridae,Orthobunyavirus,unclassified Orthobunyavirus,Orthobunyavirus
7,Calovo virus,range,#ff9f9b,Peribunyaviridae,Orthobunyavirus,unclassified Orthobunyavirus,Orthobunyavirus
8,Kaeng Khoi virus,range,#ff9f9b,Peribunyaviridae,Orthobunyavirus,None,Orthobunyavirus
9,Bunyamwera virus,range,#ff9f9b,Peribunyaviridae,Orthobunyavirus,None,Orthobunyavirus


In [186]:
write_annotation('./out/trees/Bixby_Bridge_virus_95_dedup.fasta', 'Bixby_Bridge_Virus')

,virus_name,range,color,family,genus,no_rank,clade
0,Bixby Bridge Virus,range,#fcfffd,None,None,None,None
1,sp|Q87022.2|RDRP_SCVLA RecName: Full=Probable ...,range,#fcfffd,None,None,None,None
2,Saccharomyces paradoxus virus L-A-45,range,#a1c9f4,Totiviridae,None,unclassified Totiviridae,unclassified Totiviridae
3,Saccharomyces cerevisiae virus L-A-lus,range,#ffb482,Totiviridae,Totivirus,None,Totivirus
4,Saccharomyces cerevisiae virus L-A-2,range,#ffb482,Totiviridae,Totivirus,None,Totivirus
5,Saccharomyces kudriavzevii virus L-A1,range,#a1c9f4,Totiviridae,None,unclassified Totiviridae,unclassified Totiviridae
6,Saccharomyces paradoxus virus L-A-62,range,#a1c9f4,Totiviridae,None,unclassified Totiviridae,unclassified Totiviridae
7,Saccharomyces kudriavzevii virus L-A-1082,range,#a1c9f4,Totiviridae,None,unclassified Totiviridae,unclassified Totiviridae
8,Tuber aestivum virus 1,range,#ffb482,Totiviridae,Totivirus,None,Totivirus
9,Ambrosiozyma totivirus A,range,#ffb482,Totiviridae,Totivirus,unclassified Totivirus,Totivirus


In [189]:
get_taxinfo('./out/trees/Malpaso_Creek_Virus_95_dedup.fasta')
write_annotation('./out/trees/Malpaso_Creek_Virus_95_dedup.fasta', 'Malpaso_Creek_Virus_annotation')

,virus_name,range,color,family,genus,no_rank,clade
0,Malpaso Creek Virus,range,#fcfffd,None,None,None,None
1,Araguari virus,range,#a1c9f4,Orthomyxoviridae,Thogotovirus,unclassified Thogotovirus,Thogotovirus
2,Quaranfil quaranjavirus,range,#ffb482,Orthomyxoviridae,Quaranjavirus,None,Quaranjavirus
3,Tjuloc virus,range,#8de5a1,Orthomyxoviridae,None,unclassified Orthomyxoviridae,unclassified Orthomyxoviridae
4,Wellfleet Bay virus,range,#ffb482,Orthomyxoviridae,Quaranjavirus,unclassified Quaranjavirus,Quaranjavirus
5,Johnston Atoll quaranjavirus,range,#ffb482,Orthomyxoviridae,Quaranjavirus,None,Quaranjavirus
6,Beihai orthomyxo-like virus 1,range,#ff9f9b,None,None,unclassified RNA viruses ShiM-2016,unclassified RNA viruses ShiM-2016
7,Coleopteran orthomyxo-related virus OKIAV184,range,#8de5a1,Orthomyxoviridae,None,unclassified Orthomyxoviridae,unclassified Orthomyxoviridae
8,Phasmatodean orthomyxo-related virus OKIAV172,range,#8de5a1,Orthomyxoviridae,None,unclassified Orthomyxoviridae,unclassified Orthomyxoviridae
9,Photinus pyralis orthomyxo-like virus 2,range,#8de5a1,Orthomyxoviridae,None,unclassified Orthomyxoviridae,unclassified Orthomyxoviridae


In [190]:
get_taxinfo('./out/trees/Calla_Lily_Valley_virus_95_dedup.fasta')
write_annotation('./out/trees/Calla_Lily_Valley_virus_95_dedup.fasta', '/Calla_Lily_Valley_virus')

,virus_name,range,color,family,genus,no_rank,clade
0,Calla Lily Valley Virus,range,#fcfffd,None,None,None,None
1,Fennes virus,range,#a1c9f4,None,None,unclassified Riboviria,unclassified Riboviria
2,Eyach virus,range,#ffb482,Reoviridae,Coltivirus,None,Coltivirus
3,Shelly headland virus,range,#a1c9f4,None,None,unclassified Riboviria,unclassified Riboviria
4,Tarumizu tick virus,range,#ffb482,Reoviridae,Coltivirus,None,Coltivirus
5,Colorado tick fever virus,range,#ffb482,Reoviridae,Coltivirus,None,Coltivirus
6,Kundal virus,range,#ffb482,Reoviridae,Coltivirus,None,Coltivirus
7,Lishui pangolin virus,range,#8de5a1,Reoviridae,None,unclassified Spinareovirinae,unclassified Spinareovirinae
8,Tai Forest reovirus,range,#ffb482,Reoviridae,Coltivirus,None,Coltivirus
9,Cryphonectria parasitica mycoreovirus-1 (9B21),range,#fcfffd,None,None,None,None


In [200]:
pd.set_option('display.max_rows', 200)
get_clade('./out/trees/monjiviricetes.fasta')

,virus_name,family,genus,no_rank,subfamily
0,Yankee Point Virus,None,None,None,None
1,Rocky Ridge Virus,None,None,None,None
2,Suffolk virus,Chuviridae,Mivirus,None,None
3,Lesnoe mivirus,Chuviridae,Mivirus,unclassified Mivirus,None
4,Umea virus,None,None,unclassified Mononegavirales,None
5,Genoa virus,Chuviridae,Mivirus,None,None
6,Bole Tick Virus 3,Chuviridae,Mivirus,None,None
7,Bole Tick Virus 3,Chuviridae,Mivirus,None,None
8,Liman tick virus,None,None,unclassified Riboviria,None
9,Changping Tick Virus 2,Chuviridae,Mivirus,None,None


In [281]:
write_annotation('./out/trees/Rocky_Ridge_virus_95_dedup.fasta', 'Rocky_Ridge_virus_annotation')

,virus_name,range,color,family,genus,no_rank,subfamily,clade
0,Rocky Ridge Virus,range,#eba632,None,None,None,None,None
1,Suffolk virus,range,#f77189,Chuviridae,Mivirus,None,None,Mivirus
2,Lesnoe mivirus,range,#f77189,Chuviridae,Mivirus,unclassified Mivirus,None,Mivirus
3,Umea virus,range,#639df4,None,None,unclassified Mononegavirales,None,unclassified Mononegavirales
4,Genoa virus,range,#f77189,Chuviridae,Mivirus,None,None,Mivirus
5,Bole Tick Virus 3,range,#f77189,Chuviridae,Mivirus,None,None,Mivirus
6,Liman tick virus,range,#33b16f,None,None,unclassified Riboviria,None,unclassified Riboviria
7,Changping Tick Virus 2,range,#f77189,Chuviridae,Mivirus,None,None,Mivirus
8,Karukera tick virus,range,#f77189,Chuviridae,Mivirus,None,None,Mivirus
9,Changping mivirus,range,#f77189,Chuviridae,Mivirus,None,None,Mivirus


In [17]:
pd.set_option('display.max_rows', 150)
write_annotation('./out/trees/rhabdoviridae_95_dedup.fasta', 'rhabdoviridae_annotation')

,virus_name,range,color,family,genus,no_rank,subfamily,clade
0,Doud Peak Virus,range,#eba632,None,None,None,None,None
1,Norway mononegavirus 1,range,#f77189,None,None,unclassified Mononegavirales,None,unclassified Mononegavirales
2,Mononegavirales sp.,range,#f77189,None,None,unclassified Mononegavirales,None,unclassified Mononegavirales
3,Manly virus,range,#639df4,None,None,unclassified Riboviria,None,unclassified Riboviria
4,Huangpi Tick Virus 3,range,#33b16f,Rhabdoviridae,None,unclassified Rhabdoviridae,None,unclassified Rhabdoviridae
5,Tacheng Tick Virus 3,range,#33b16f,Rhabdoviridae,None,unclassified Rhabdoviridae,None,unclassified Rhabdoviridae
6,Bole Tick Virus 2,range,#edd403,Rhabdoviridae,Alpharicinrhavirus,None,Alpharhabdovirinae,Alpharicinrhavirus
7,Wuhan Tick Virus 1,range,#edd403,Rhabdoviridae,Alpharicinrhavirus,None,Alpharhabdovirinae,Alpharicinrhavirus
8,Blanchseco virus,range,#edd403,Rhabdoviridae,Alpharicinrhavirus,None,Alpharhabdovirinae,Alpharicinrhavirus
9,Xinzhou dimarhabdovirus virus 1,range,#f3c2f4,None,None,unclassified RNA viruses ShiM-2016,None,unclassified RNA viruses ShiM-2016


In [15]:
tax_dict

{'2518294': {'superkingdom': 'Viruses',
  'clade': 'Riboviria',
  'kingdom': 'Orthornavirae',
  'phylum': 'Lenarviricota',
  'class': 'Amabiliviricetes',
  'order': 'Wolframvirales',
  'family': 'Narnaviridae',
  'no rank': 'unclassified Narnaviridae'},
 '186772': {'superkingdom': 'Viruses',
  'clade': 'Riboviria',
  'kingdom': 'Orthornavirae',
  'phylum': 'Lenarviricota',
  'class': 'Amabiliviricetes',
  'order': 'Wolframvirales',
  'family': 'Narnaviridae',
  'genus': 'Narnavirus'},
 '2662286': {'superkingdom': 'Viruses',
  'clade': 'Riboviria',
  'kingdom': 'Orthornavirae',
  'phylum': 'Lenarviricota',
  'class': 'Amabiliviricetes',
  'order': 'Wolframvirales',
  'family': 'Narnaviridae',
  'genus': 'Narnavirus',
  'no rank': 'unclassified Narnavirus'},
 '2716636': {'superkingdom': 'Viruses',
  'clade': 'Riboviria',
  'kingdom': 'Orthornavirae',
  'phylum': 'Lenarviricota',
  'class': 'Amabiliviricetes',
  'order': 'Wolframvirales',
  'family': 'Narnaviridae',
  'no rank': 'unclassi